In [18]:
import numpy as np
import dgl
import scanpy as sc
import pandas as pd

import torch 
import torch.nn as nn
import torch.nn.functional as F

In [19]:
adata = sc.read('/home/ngr4/project/sccovid/data/processed/hbec_scv2.h5ad')

In [24]:
adata_test = adata[adata.obs['split']=='test', :]

/gpfs/ysm/project/dijk/ngr4/conda_envs/py38dev/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [40]:
g = dgl.from_scipy(adata_test.obsp['connectivities'])
g.ndata['x'] = adata_test.X

AttributeError: device not found

In [41]:
adata_test.X

<11647x24714 sparse matrix of type '<class 'numpy.float32'>'
	with 33740370 stored elements in Compressed Sparse Row format>

In [4]:
dataset = dgl.data.CoraGraphDataset()

Extracting file to /home/ngr4/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [7]:
g = dataset[0]

In [10]:
from dgl.nn import GraphConv

In [13]:
class GCN(nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super().__init__()
        self.conv1 = GraphConv(in_dim, h_dim)
        self.conv2 = GraphConv(h_dim, out_dim)
    def forward(self, g, in_dim):
        h = self.conv1(g, in_dim)
        h = F.elu(h)
        return self.conv2(g, h)
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [14]:
model

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [15]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.192 (best 0.192), test acc: 0.153 (best 0.153)
In epoch 5, loss: 1.863, val acc: 0.530 (best 0.538), test acc: 0.553 (best 0.548)
In epoch 10, loss: 1.763, val acc: 0.578 (best 0.578), test acc: 0.540 (best 0.540)
In epoch 15, loss: 1.641, val acc: 0.654 (best 0.654), test acc: 0.662 (best 0.662)
In epoch 20, loss: 1.498, val acc: 0.714 (best 0.716), test acc: 0.735 (best 0.726)
In epoch 25, loss: 1.339, val acc: 0.728 (best 0.728), test acc: 0.715 (best 0.715)
In epoch 30, loss: 1.169, val acc: 0.736 (best 0.736), test acc: 0.720 (best 0.722)
In epoch 35, loss: 0.997, val acc: 0.736 (best 0.738), test acc: 0.726 (best 0.716)
In epoch 40, loss: 0.834, val acc: 0.742 (best 0.744), test acc: 0.731 (best 0.729)
In epoch 45, loss: 0.685, val acc: 0.738 (best 0.744), test acc: 0.735 (best 0.729)
In epoch 50, loss: 0.557, val acc: 0.740 (best 0.744), test acc: 0.743 (best 0.729)
In epoch 55, loss: 0.450, val acc: 0.748 (best 0.748), test acc: 0.744 (best 0